In [1]:
import pandas as pd
import numpy as np

ruta_archivo = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Scripts\Ciclo\SI_SO Ciclo Sep-2024AE.csv"
df = pd.read_csv(ruta_archivo, delimiter=',', decimal=',', encoding='utf-8').rename(columns={'Canal': 'Canal 3', 'Cantidad': 'Venta', 'idSKU': 'Ultimo Eslabón'})
df['ID'] = df['Ultimo Eslabón'].astype(str) + df['Canal 3'].astype(str)
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%Y-%m-%d')
df = df.sort_values(by=['Ultimo Eslabón', 'Fecha']).reset_index(drop=True)
pivot_table = df.pivot_table(index=['Ultimo Eslabón', 'Canal 3'], columns='Fecha', values='Venta', fill_value=0).reset_index()
pivot_table = pivot_table[pivot_table.iloc[:, 2:].sum(axis=1) >= 2]
ultima_fecha = pivot_table.columns[-1]
primer_fecha = ultima_fecha - pd.DateOffset(months=23)
ventas_ultimos_24_meses = pivot_table.loc[:, primer_fecha:ultima_fecha]
intervalos = ventas_ultimos_24_meses.apply(lambda x: (x != 0).astype(int).diff().fillna(1).abs().sum(), axis=1)
ventas_activas = (ventas_ultimos_24_meses != 0).sum(axis=1)
pivot_table['ADI'] = intervalos / ventas_activas
ventas_no_cero = ventas_ultimos_24_meses.replace(0, np.nan)
pivot_table['CV²'] = (ventas_no_cero.std(axis=1) / ventas_no_cero.mean(axis=1)) ** 2
def clasificar_demanda(row):
    if row['ADI'] < 1.32 and row['CV²'] < 0.49: return 'Smooth'
    elif row['ADI'] >= 1.32 and row['CV²'] < 0.49: return 'Intermittent'
    elif row['ADI'] < 1.32 and row['CV²'] >= 0.49: return 'Erratic'
    else: return 'Lumpy'
pivot_table['Demand Type'] = pivot_table.apply(clasificar_demanda, axis=1)
melted_data = pivot_table.melt(id_vars=['Ultimo Eslabón', 'Canal 3', 'ADI', 'CV²', 'Demand Type'], var_name='Fecha', value_name='Venta')
melted_data['Fecha'] = pd.to_datetime(melted_data['Fecha'], format='%Y-%m-%d')
data = melted_data
data.shape

(4071502, 7)

In [2]:
data = melted_data[melted_data['Demand Type'] == 'Smooth']
data = data[['Ultimo Eslabón', 'Canal 3', 'Fecha', 'Venta']]
data.head(2)

,Ultimo Eslabón,Canal 3,Fecha,Venta
7,100481,CL RETAIL,2018-01-01,63.0
10,100485,CL RETAIL,2018-01-01,0.0


In [4]:
import pandas as pd
import numpy as np
from prophet import Prophet
import os

data['ID'] = data.apply(lambda row: f"{row['Ultimo Eslabón']}_{row['Canal 3']}", axis=1)
data['Fecha'] = pd.to_datetime(data['Fecha'], format='%Y-%m-%d')

predictions = []

for id_value in data['ID'].unique():
    data_id = data[data['ID'] == id_value][['Fecha', 'Venta']].rename(columns={'Fecha': 'ds', 'Venta': 'y'})
    
    if data_id['y'].isnull().any() or np.isinf(data_id['y']).any():
        print(f'Serie temporal con valores nulos o infinitos para ID: {id_value}')
        continue
    
    try:
        model = Prophet()
        model.fit(data_id)
        
        future_dates = pd.date_range(start='2024-01-01', periods=12, freq='MS').to_frame(index=False, name='ds')
        forecast = model.predict(future_dates)
        
        for date, pred in zip(forecast['ds'], forecast['yhat']):
            predictions.append({
                'ID': id_value,
                'Fecha': date.strftime('%Y-%m-%d'),
                'Prediccion_Venta': pred
            })
    except Exception as e:
        print(f'Error al ajustar el modelo para ID: {id_value} - {e}')
        continue

predictions_df = pd.DataFrame(predictions)
output_dir = r'C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Scripts\Ciclo'
output_file = os.path.join(output_dir, 'PROPHET_SMOOTH_DIC.csv')
predictions_df.to_csv(output_file, index=False, sep=';', decimal=',')

print(f'Predicciones guardadas en {output_file}')


22:35:19 - cmdstanpy - INFO - Chain [1] start processing
22:35:19 - cmdstanpy - INFO - Chain [1] done processing
22:35:20 - cmdstanpy - INFO - Chain [1] start processing
22:35:20 - cmdstanpy - INFO - Chain [1] done processing
22:35:20 - cmdstanpy - INFO - Chain [1] start processing
22:35:20 - cmdstanpy - INFO - Chain [1] done processing
22:35:20 - cmdstanpy - INFO - Chain [1] start processing
22:35:21 - cmdstanpy - INFO - Chain [1] done processing
22:35:21 - cmdstanpy - INFO - Chain [1] start processing
22:35:21 - cmdstanpy - INFO - Chain [1] done processing
22:35:21 - cmdstanpy - INFO - Chain [1] start processing
22:35:21 - cmdstanpy - INFO - Chain [1] done processing
22:35:22 - cmdstanpy - INFO - Chain [1] start processing
22:35:22 - cmdstanpy - INFO - Chain [1] done processing
22:35:22 - cmdstanpy - INFO - Chain [1] start processing
22:35:22 - cmdstanpy - INFO - Chain [1] done processing
22:35:22 - cmdstanpy - INFO - Chain [1] start processing
22:35:23 - cmdstanpy - INFO - Chain [1]

Predicciones guardadas en C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Scripts\Ciclo\PROPHET_SMOOTH_DIC.csv
